<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/VGG19_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import keras 
from keras.applications import VGG19
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

In [4]:
conv_base = VGG19(weights='imagenet', include_top = False, input_shape=(300,300,3))

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [7]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [8]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [9]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [10]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,512))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)


In [11]:
samples = samples.reshape(nos,9*9*512)

In [12]:
samples.shape, labels.shape

((472, 41472), (472,))

In [31]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 1s 1ms/step - loss: 2.4254 - accuracy: 0.6077 - val_loss: 1.0512 - val_accuracy: 0.7288
Epoch 2/100
413/413 [==============================] - 0s 957us/step - loss: 0.6141 - accuracy: 0.8136 - val_loss: 0.5468 - val_accuracy: 0.6949
Epoch 3/100
413/413 [==============================] - 0s 966us/step - loss: 0.2909 - accuracy: 0.8814 - val_loss: 0.4512 - val_accuracy: 0.7627
Epoch 4/100
413/413 [==============================] - 0s 906us/step - loss: 0.2065 - accuracy: 0.9225 - val_loss: 0.4113 - val_accuracy: 0.7966
Epoch 5/100
413/413 [==============================] - 0s 916us/step - loss: 0.1731 - accuracy: 0.9346 - val_loss: 0.4942 - val_accuracy: 0.7627
Epoch 6/100
413/413 [==============================] - 0s 924us/step - loss: 0.1466 - accuracy: 0.9637 - val_loss: 0.3943 - val_accuracy: 0.8475
Epoch 7/100
413/413 [==============================] - 0s 913us/step - loss: 0.1

In [32]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.7803160919540231 0.8349137931034484


Training on whole Training Dataset

In [26]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [17]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [18]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [19]:
datagentest.fit(xtest)

In [20]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,512))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 512)


In [21]:
test_samples = test_samples.reshape(testnos,9*9*512)

In [27]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 2ms/step - loss: 1.9837 - accuracy: 0.6504 - val_loss: 2.2474 - val_accuracy: 0.4457
Epoch 2/100
472/472 [==============================] - 1s 1ms/step - loss: 0.5674 - accuracy: 0.8114 - val_loss: 0.8082 - val_accuracy: 0.6377
Epoch 3/100
472/472 [==============================] - 1s 1ms/step - loss: 0.3110 - accuracy: 0.8644 - val_loss: 0.3846 - val_accuracy: 0.8388
Epoch 4/100
472/472 [==============================] - 1s 1ms/step - loss: 0.2104 - accuracy: 0.9195 - val_loss: 0.5334 - val_accuracy: 0.7717
Epoch 5/100
472/472 [==============================] - 1s 1ms/step - loss: 0.2017 - accuracy: 0.9216 - val_loss: 0.5433 - val_accuracy: 0.7772
Epoch 6/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1818 - accuracy: 0.9153 - val_loss: 0.6767 - val_accuracy: 0.7373
Epoch 7/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1316 - accuracy: 0.

In [28]:
test_preds = model.predict(test_samples, batch_size = 20)

In [29]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [30]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.6101694915254238, 0.8661257606490872)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/